# 0. what for
1. 对大数据集计算np
# 1. Preparations
## 1.1 全局设置

In [28]:
#导入pandas包
import pandas as pd
import numpy as np

auctions_cleaned_path = r'E:\DATA\large_dta\auctions_cleaned.csv'   # retail修正的data

# Output
data_np_path = r'E:\DATA\large_dta\large_auctions_np.csv'                 #
settings_np_path = r'E:\DATA\large_dta\large_settings_np.csv'             #
prod_path = r'E:\DATA\large_dta\large_prod.csv'
# settings path
settings_GT_path = r'E:\DATA\large_dta\large_settings_GT.csv'
settings_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'

- 读取

In [29]:
data = pd.read_csv(auctions_cleaned_path,encoding="utf-8")
# features里添加`flg_fixedprice`，以区分两种auction
unique_setting_GT = ['bidincrement','bidfee','retail','flg_fixedprice']
unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_fixedprice']

In [30]:
threshold = 16

# 2. threshold
1. threshold筛选的是某个unique_setting_NN下，拍卖发生的次数不够多，那么这个setting不具有代表性
2.

In [31]:
data.head()

,auction_id,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,N,bidincrement,bidfee,retail,desc
0,138823,1,0,Consumer Goods,Euro,1.3978,9.546973,10.0,0.01,0.01,10.0,COSMETICS Brush SET 6 PIECES MADE OF WOOD
1,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
2,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
3,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER
4,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER


- 计算cnt_uniq: 这个uniq settings下一共出现了多少场auctions


In [32]:
features = unique_setting_NN

In [33]:
data_NN = data.groupby(by = features,as_index=False)
# cnt_total = data_NN.endtimeperiod.sum()
# cnt_total
data_NN_grouped = pd.DataFrame(data_NN.size())
data_NN_grouped.rename(columns={'size': 'cnt_uniq'}, inplace=True)
print(data_NN_grouped.head())
data_NN_grouped.shape

                         desc  bidincrement  bidfee  retail  flg_fixedprice  \
0             1 KG SILVER BAR          0.01    0.01   559.0               0   
1       10 GRAMS OF GOLD BARS          0.01    0.01   325.0               0   
2      10 GRAMS PLATINUM BARS          0.01    0.01   366.0               0   
3  10 MARKS BAVARIA GOLDMüNZE          0.01    0.01   268.0               0   
4      1000 FREEBIDS VOUCHER!          0.01    0.01   671.0               0   

   cnt_uniq  
0        33  
1        21  
2         7  
3         5  
4         3  


(773, 6)

- cnt_uniq加入大表，筛下去那些小于threshold的settings

In [34]:
data_cnt = pd.merge(data,data_NN_grouped,on=features,how="left")
data_thrld = data_cnt[ data_cnt.cnt_uniq>=threshold ].copy()

settings_NN_thrld = data_thrld[features].drop_duplicates(inplace=False)
settings_NN_thrld.reset_index(drop=True,inplace=True)
n_settings = settings_NN_thrld.shape[0]
print(f"当threshold = *{threshold}* 时，剩余{n_settings}个 auction settings")

当threshold = *16* 时，剩余80个 auction settings


In [35]:
settings_NN_thrld.to_csv(settings_np_path,encoding="utf-8",index=False,header=True)
settings_NN_thrld.head()

,desc,bidincrement,bidfee,retail,flg_fixedprice
0,20 BIDS VOUCHER,0.01,0.01,14.0,0
1,20 BIDS VOUCHER,0.01,0.01,15.0,0
2,50 BIDS VOUCHER,0.01,0.01,35.0,0
3,50 FREEBIDS VOUCHER!,0.01,0.01,35.0,0
4,CLATRONIC PM 2658 HOT AIR POPCORN MAKER (WHITE),0.01,0.01,35.0,0


In [36]:
data_thrld.describe()

,auction_id,pennyauction,flg_fixedprice,exchangerate,adjvalue,N,bidincrement,bidfee,retail,cnt_uniq
count,8872.00000,8872.0,8872.0,8872.000000,8872.000000,8872.000000,8.872000e+03,8.872000e+03,8872.000000,8872.000000
mean,154073.35381,1.0,0.0,1.331190,586.631957,2190.529869,1.000000e-02,1.000000e-02,719.749098,382.694545
std,22173.37520,0.0,0.0,0.056136,507.141788,3367.467505,9.558865e-16,9.558865e-16,583.842005,323.425343
min,33894.00000,1.0,0.0,1.249100,13.305000,1.000000,1.000000e-02,1.000000e-02,14.000000,16.000000
25%,143100.75000,1.0,0.0,1.289200,134.412750,212.000000,1.000000e-02,1.000000e-02,113.000000,134.000000
50%,157712.50000,1.0,0.0,1.324700,412.160100,919.000000,1.000000e-02,1.000000e-02,690.000000,258.000000
75%,171799.25000,1.0,0.0,1.359300,1022.735000,2744.250000,1.000000e-02,1.000000e-02,1306.000000,469.000000
max,182400.00000,1.0,0.0,1.594300,2064.500700,37715.000000,1.000000e-02,1.000000e-02,2142.000000,1033.000000


# 3. 计算np
## 3.1 计算 'n'
1. 'N'这一列已经记录了这个auction的duration

In [37]:
data_thrld.head()

,auction_id,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,N,bidincrement,bidfee,retail,desc,cnt_uniq
1,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377
2,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377
3,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER,377
4,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER,377
5,175995,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0,20 BIDS VOUCHER,377


## 3.2 计算'P'
### 3.2.1 计算'cnt_N'
1. 应该计算当前setting下，不同的‘N’值出现的次数'cnt_N'

In [38]:
features = unique_setting_NN+['N']

data_grouped = data_thrld.groupby(by=features,as_index=False)
data_cnt_N = pd.DataFrame(data_grouped.size())
data_cnt_N.rename(columns={'size': 'cnt_N'}, inplace=True)

data_P = pd.merge(data_thrld,data_cnt_N,on=features,how="left")
data_P.head()
# pd.merge(data_thrld,data_cnt_N,on=features,how="")

,auction_id,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,N,bidincrement,bidfee,retail,desc,cnt_uniq,cnt_N
0,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,10
1,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,10
2,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,15
3,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,6
4,175995,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,6


### 3.2.2 计算P

In [39]:
data_P['P'] = data_P['cnt_N'] / data_P['cnt_uniq']
data_P.head()

,auction_id,pennyauction,flg_fixedprice,type,currency,exchangerate,adjvalue,N,bidincrement,bidfee,retail,desc,cnt_uniq,cnt_N,P
0,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,10,0.026525
1,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,10,0.026525
2,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,15,0.039788
3,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,6,0.015915
4,175995,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0,20 BIDS VOUCHER,377,6,0.015915


In [40]:
features_keep = unique_setting_NN +['N'] + ['P']+['cnt_N']  # 想保留的列
data_P = data_P[features_keep].copy()
data_P.head()

,desc,bidincrement,bidfee,retail,flg_fixedprice,N,P,cnt_N
0,20 BIDS VOUCHER,0.01,0.01,14.0,0,18.0,0.026525,10
1,20 BIDS VOUCHER,0.01,0.01,14.0,0,18.0,0.026525,10
2,20 BIDS VOUCHER,0.01,0.01,14.0,0,15.0,0.039788,15
3,20 BIDS VOUCHER,0.01,0.01,14.0,0,32.0,0.015915,6
4,20 BIDS VOUCHER,0.01,0.01,14.0,0,16.0,0.015915,6


# 4. save
- 保存data
- 保存item方便embedding

In [45]:
data_P.to_csv(data_np_path,encoding="utf-8",index=False,header=True)
print(f"小数据集一共有 {data_P.shape[0]} 条记录")

小数据集一共有 8872 条记录


In [42]:
item_df = pd.DataFrame(data_P['desc'].drop_duplicates(inplace=False))
item_df.reset_index(drop=True,inplace=True)
item_df.to_csv(prod_path,encoding="utf-8",index=False,header=True)
item_df.head()

,desc
0,20 BIDS VOUCHER
1,50 BIDS VOUCHER
2,50 FREEBIDS VOUCHER!
3,CLATRONIC PM 2658 HOT AIR POPCORN MAKER (WHITE)
4,MAKITA CORDLESS ARTICULATED WRENCH


- 保存data key

In [43]:
data_auction_GT = data_P[unique_setting_GT].copy()
settings_GT = data_auction_GT.drop_duplicates(ignore_index=True)
settings_GT.to_csv(settings_GT_path,encoding="utf-8",index=False,header=True)

print(f"当setting = {unique_setting_GT}")
print(f"一共有 *{settings_GT.shape[0]}*场不同的auction")

当setting = ['bidincrement', 'bidfee', 'retail', 'flg_fixedprice']
一共有 *75*场不同的auction


In [44]:
data_auction_NN = data_P[unique_setting_NN].copy()
settings_NN = data_auction_NN.drop_duplicates(ignore_index=True)
settings_NN.to_csv(settings_NN_path,encoding="utf-8",index=False,header=True)

print(f"当setting = {unique_setting_NN}")
print(f"一共有 *{settings_NN.shape[0]}*场不同的auction")

当setting = ['desc', 'bidincrement', 'bidfee', 'retail', 'flg_fixedprice']
一共有 *80*场不同的auction
